In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [1]:
# !pip install dash
# !pip install jupyter-dash
# !pip install plotly
# !pip install dash-bootstrap-components
# !pip install dash-ag-grid

In [ ]:
import sys
# Importamos nuestras propias funciones
sys.path.append('drive/MyDrive/TFG/src/')

In [ ]:
from numpy import array, concatenate
import keras
from keras.layers import TimeDistributed, Conv1D, MaxPooling1D, Flatten, Dense, LSTM, Dropout, RepeatVector
from sklearn.metrics import  mean_absolute_error, r2_score
from keras.utils import plot_model
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from data_preparation import train_test_split, scale
from supervised_learning import to_supervised, prior_inverse, invert_scale
from evaluation import measure_rmse, summarize_scores
from data_visualization import boxplot_errors, plot_loss, plot_metrics, plot_pred_vs_real, residuals_dist, residuals_vs_pred

# fit a model
def model_fit(train_x, train_y, test_x, test_y, config):
  # unpack config
  n_steps, n_filters, n_kernel, n_nodes, n_epochs, n_batch, dropout = config

  # Prepare data
  n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[0]

  early_stopping = EarlyStopping(patience=5, monitor='loss')
  # define model
  model = keras.Sequential()
  model.add(Conv1D(filters=n_filters, kernel_size=n_kernel,
                                   activation='relu', input_shape=(n_timesteps, n_features)))
  model.add(Conv1D(filters=n_filters, kernel_size=n_kernel,
                                   activation='relu'))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Flatten())
  model.add(Dense(n_nodes, activation='relu'))
  model.add(Dense(1))

  model.compile(loss='mse', optimizer='adam', metrics=['mae'])
  # fit
  history = model.fit(train_x, train_y, epochs=n_epochs, batch_size=n_batch, verbose=0, callbacks=[early_stopping],  validation_data = (test_x, test_y))
  # print(model.summary())
  return model, history


In [ ]:
import pandas as pd
import time
# Datos totales de energía fotovoltaica
df = pd.read_csv('drive/MyDrive/TFG/data/PV_data.csv', index_col='Time')

# Datos primer año
divide = '2020-01-01'
first_year = df.loc[:divide]

start_test = '2019-09-31'

config = [24, 32, 2, 30, 30, 150, 0.2]

n_steps, _, _, _, _, _, _ = config

train, test = train_test_split(first_year, start_test)

# prepare data
train = train.to_numpy()
test = test.to_numpy()

# Escalamos entre 0 y 1
scaler, train_scaled, test_scaled = scale(train, test, 0, 1)

# Dividimos en arrays las valores de entrenamiento y test
train_x, train_y = to_supervised(train_scaled, n_steps)
test_x, test_y = to_supervised(test_scaled, n_steps)

model, history = model_fit(train_x, train_y, test_x, test_y, config)

In [ ]:
from dash import dcc, html, Dash
import dash_bootstrap_components as dbc
import dash_ag_grid as dag
from dash.dependencies import Input, Output, State
import plotly.express as px
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.graph_objects as go

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    html.H1("Predicción de Generación de Energía Solar Fotovoltaica", style={'color': '#004d40', 'textAlign': 'center', 'marginTop': '20px'}),

    html.H2("Selección valores fecha sesión mercado intradiario", style={'color': '#004d40', 'textAlign': 'center', 'marginTop': '20px'}),

    html.Div([
        html.Div([
            html.Div([
                html.Label('Año', style={'fontSize': 24, 'color': '#004d40'}),
                dcc.Dropdown(
                    id='dropdown-año',
                    options=[{'label': str(i), 'value': i} for i in range(2019, 2021)],
                    placeholder='Selecciona',
                    style={'width': '100%', 'textAlign': 'left'}
                )
            ], style={'width': '20%', 'display': 'inline-block', 'padding': '10px'}),

            html.Div([
                html.Label('Mes', style={'fontSize': 24, 'color': '#004d40'}),
                dcc.Dropdown(
                    id='dropdown-mes',
                    options=[{'label': str(i), 'value': i} for i in range(1, 13)],
                    placeholder='Selecciona',
                    style={'width': '100%', 'textAlign': 'left'}
                )
            ], style={'width': '20%', 'display': 'inline-block', 'padding': '10px'}),

            html.Div([
                html.Label('Día', style={'fontSize': 24, 'color': '#004d40'}),
                dcc.Dropdown(
                    id='dropdown-dia',
                    options=[{'label': str(i), 'value': i} for i in range(1, 32)],
                    placeholder='Selecciona',
                    style={'width': '100%', 'textAlign': 'left'}
                )
            ], style={'width': '20%', 'display': 'inline-block', 'padding': '10px'}),

            html.Div([
                html.Label('Hora', style={'fontSize': 24, 'color': '#004d40'}),
                dcc.Dropdown(
                    id='dropdown-hora',
                    options=[{'label': str(i), 'value': i} for i in range(0, 24)],
                    placeholder='Selecciona',
                    style={'width': '100%', 'textAlign': 'left'}
                )
            ], style={'width': '20%', 'display': 'inline-block', 'padding': '10px'}),
        ], style={'display': 'flex', 'justifyContent': 'space-around'}),
    ]),

    html.Div([
        dbc.Button("Predecir Generación", id='predict-button', color='success', style={'fontSize': '18px', 'padding': '12px 25px'})
    ], style={'textAlign': 'center', 'marginTop': '20px'}),

    html.H2("La generación la próxima hora será de:", style={'textAlign': 'center', 'color': '#004d40', 'marginTop': '40px'}),

    html.Div(id='prediction-output', style={'fontSize': 40, 'textAlign': 'center', 'color': '#004d40', 'marginTop': '30px'})

], style={'backgroundColor': '#e8f5e9', 'padding': '30px', 'borderRadius': '10px', 'minHeight': '100vh'})

# Definir la callback para actualizar la predicción
@app.callback(
    Output('prediction-output', 'children'),
    Input('predict-button', 'n_clicks'),
    State('dropdown-año', 'value'),
    State('dropdown-mes', 'value'),
    State('dropdown-dia', 'value'),
    State('dropdown-hora', 'value')
)


def predict_generation(n_clicks, anyo, mes, dia, hora):
    if n_clicks is None:
        return ""
    df = pd.read_csv('drive/MyDrive/TFG/data/PV_data.csv', index_col='Time')

    # Datos primer año
    divide = '2020-01-01'
    first_year = df.loc[:divide]

    df_test = first_year[start_test:]
    fecha_hora = datetime(anyo, mes, dia, hora)
    mask = fecha_hora.strftime("%Y-%m-%d %H:%M:%S")
    df_filter_test = df_test.loc[:mask]

    # Escalamos entre 0 y 1
    scaler, train_scaled, test_scaled = scale(df_filter_test.to_numpy(), df_filter_test.to_numpy(), 0, 1)

    test_x, test_y = to_supervised(test_scaled, n_steps)
    res = model.predict(test_x)
    inv_pred = res[-1][0]

    max = df_filter_test['Power (MW)'].max()
    min = df_filter_test['Power (MW)'].min()
    pred = inv_pred * (max-min) + min

    return f"{pred:.2f} MW"

# Ejecutar la aplicación
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>